## EDA - Scripts

### Import Statements

In [4]:
%matplotlib inline
from bs4 import BeautifulSoup
import requests
import pandas as pd
import lifelines
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import seaborn as sns
sns.set(style="whitegrid", color_codes=True)

#### Closest match from list of words

In [ ]:
list_crimes = list(set(list_crimes))
import difflib
for index,word in enumerate(list_crimes):
    print word,':',difflib.get_close_matches(word, list_crimes[index+1:],20)

#### Code Name Map

In [ ]:
distict_code_name_map = {
'A1' : 'Downtown','A15' : 'Charlestown','A7' :'East Boston',
'B2' : 'Roxbury',
'B3' : 'Mattapan',
'C6' : 'South Boston',
'C11' : 'Dorchester',
'D4' : 'South End',
'D14' : 'Brighton',
'E5' :'West Roxbury',
'E13' : 'Jamaica Plain',
'E18': 'Hyde Park',
'HTU': 'Human Traffic Unit'
}

#### If - Else List Comprehension

In [ ]:
list_hours = list()
for row in df_boston.times:
    list_hours 
    if 'AM' in row:
        list_hours.append(int(row.split(':',1)[0]))
    else:
        list_hours.append(int(row.split(':',1)[0]) + 12 )
df_boston['hours'] = pd.Series(list_hours)

In [ ]:
list_hours = [ int(row.split(':',1)[0]) if 'AM' in row else int(row.split(':',1)[0]) + 12 for row in df_boston.times]

In [ ]:
table = ''.join(chr(index) if index in ords_to_keep else replace_with
                for index in xrange(15))

#### If List Comprehension

In [ ]:
outputlist = []
for y in a:
    if y not in b:
        outputlist.append(y)

In [ ]:
outputlist = [y for y in a if y not in b]

#### Drop Multiple Columns

In [ ]:
df_boston = df_boston.drop(['compnos','naturecode'],1)

#### IPython AutoTime

In [ ]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

In [ ]:
x = [i for i in range(0,100)]
x.sort()

#### Proportion of Not-Null values for Series 

In [ ]:
sum(df_gtd.approxdate.notnull()) / float(len(df_gtd))

#### Remove those features with more than x % null values

In [ ]:
for col in df_gtd.columns:
    rate = sum(df_gtd[col].notnull())/float(len(df_gtd)) * 100
    if rate < 1:
        df_gtd = df_gtd.drop(col,1)

#### Imputing Missing Values: 
Columns of datatype object are replaced by the most frequent value while the missing values in the column of other datatypes are replaced by their median.
Source: http://stackoverflow.com/questions/25239958/impute-categorical-missing-values-in-scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):
    def __init__(self):
        """Impute missing values.
        Columns of dtype object are imputed with the most frequent value 
        in column.
        Columns of other types are imputed with median of column.
        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self
    
    def transform(self, X, y=None):
        return X.fillna(self.fill)

#### Save dataframe to pickle

In [ ]:
#Saving these to pickle
df_gtd.to_pickle('df_gtd.pkl')

#### Animated Plotting

The animated plotting is used from Jake's post on embedding animated plots in IPython notebooks:
http://jakevdp.github.io/blog/2013/05/12/embedding-matplotlib-animations/

In [ ]:
from matplotlib import animation, pyplot as plt
from tempfile import NamedTemporaryFile

VIDEO_TAG = """<video controls autoplay>
 <source src="data:{0}">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.m4v') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264', '-pix_fmt', 'yuv420p'])
            video = open(f.name, "rb").read()
        anim._encoded_video = 'video/mp4;base64,' + video.encode("base64")
    # prevent figure displayed as a PNG below the animation
    plt.close()
    
    return VIDEO_TAG.format(anim._encoded_video)

animation.Animation._repr_html_ = anim_to_html

In [ ]:
import matplotlib.animation as animation
import random
def init():
    line.set_data([], [])
    return line,

def calc_parameters(i):
    alpha_param = 81
    beta_param = 219
    for j in range(0,i):
        if l[j] is 'H':
            alpha_param = alpha_param + 1
        else:
            beta_param =  beta_param + 1    
    return alpha_param, beta_param

def animate(i):
    alpha_param, beta_param = calc_parameters(i)
    if l[i] is 'H':
        alpha_param = alpha_param + 1
    else:
        beta_param =  beta_param + 1
    beta_dist = beta(alpha_param, beta_param)
    y = beta_dist.pdf(x)
    line.set_data(x, y)
    return line,alpha_param, beta_param

def randomly(seq):
    shuffled = list(seq)
    random.shuffle(shuffled)
    return shuffled

fig = plt.figure()
ax = plt.axes(xlim=(0.15, 0.45), ylim = (0,26))
x = np.linspace(0,1,600)
alpha_param, beta_param = calc_parameters(0)
beta_dist = beta(alpha_param, beta_param)

y = beta_dist.pdf(x)
line, = ax.plot(x, y, lw=2, label='beta_posterior')
legend = ax.legend(loc='upper right', shadow=True, fontsize='x-small')
l = randomly('H'* 200 + 'T' * 400)
list_means = []
    
animation.FuncAnimation(fig, animate, init_func=init, frames=600, interval=20, blit=True)

#### Scrape and store in DataFrame - Example

In [ ]:
def scrape_data(page_count):
    """
    scrapes the required data present in the form of a table from the given url
    :param url: page of the ESPNCricinfo Stats URL query
    :return: the raw unicode text
    """
    url = statsguru_query_url
    complete_url = url.partition('page=1')[0] + "page=" + str(page_count) + url.partition('page=1')[-1]
    r = requests.get(complete_url)
    data = r.text
    soup = BeautifulSoup(data)
    table = soup.find_all('table')
    return table[2].text

#### Transforming data - Example

In [ ]:
def create_data():
    """
    scrapes the data, cleans it and transforms the data to load into a pandas dataframe
    """
    page_count = 12
    df = pd.DataFrame()
    get_list_columns = lambda text, start_index, end_index: [str(unicode_text)
                                                                for unicode_text in text][start_index:end_index]
    get_data_rows = lambda text, start_index: text[start_index:]
    remove_all_occurences = lambda data, item: [x for x in data if x != item]
    get_list_rows = lambda data: [data[index: index + 13] for index, row in enumerate(data) if index % 13 == 0]
    for page in range(1, page_count + 1):
        raw_text = scrape_data(page)
        clean_text = clean_data(raw_text)
        list_columns = get_list_columns(clean_text, 3, 16)
        data_rows = get_data_rows(clean_text, 16)
        data_rows = remove_all_occurences(data_rows, u'')
        list_rows = get_list_rows(data_rows)
        df_new = pd.DataFrame(list_rows, columns=list_columns)
        if len(df) == 0:
            df = df_new
        else:
            df = pd.concat([df, df_new])
    return df

#### Lambda Function - Example

In [ ]:
calc_career_length = lambda df: [row.career_end_date - row.career_start_date + 1 for index, row in df.iterrows()]
df_span['career_length'] = calc_career_length(df_span)

#### Replace null with zeros

In [ ]:
df_boston.fillna(0.0);

#### Remove rows whose column equal to particular value

In [ ]:
df_boston = df_boston[df_boston.latitudes != 0.0]
df_boston = df_boston[df_boston.longitudes != 0.0]

#### Set pandas show all columns in DataFrame

In [ ]:
pd.set_option('display.max_columns', None)

#### GroupBy Example

In [ ]:
def f(x):
     return Series(dict(Number_of_tweets = x['content'].count(), 
                        Company=x['Company'].min(),
                        Description=x['from_user_description'].min(),
                        ))

In [ ]:
account_count = df.groupby('from_user_screen_name').apply(f)

#### adding data to sqlite in chunks

In [ ]:
from sqlalchemy import create_engine # database connection
disk_engine = create_engine('sqlite:///311_8M.db') # Initializes database with filename 311_8M.db in current directory
start = dt.datetime.now()
chunksize = 20000
j = 0
index_start = 1

for df in pd.read_csv('311_100M.csv', chunksize=chunksize, iterator=True, encoding='utf-8'):
    
    df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) # Remove spaces from columns

    df['CreatedDate'] = pd.to_datetime(df['CreatedDate']) # Convert to datetimes
    df['ClosedDate'] = pd.to_datetime(df['ClosedDate'])

    df.index += index_start

    # Remove the un-interesting columns
    columns = ['Agency', 'CreatedDate', 'ClosedDate', 'ComplaintType', 'Descriptor',
               'CreatedDate', 'ClosedDate', 'TimeToCompletion',
               'City']

    for c in df.columns:
        if c not in columns:
            df = df.drop(c, axis=1)    

    
    j+=1
    print '{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize)

    df.to_sql('data', disk_engine, if_exists='append')
    index_start = df.index[-1] + 1
